In [110]:
import pandas as pd
import csv
from datetime import datetime
mark = datetime.now()
att_file = 'DeviceLogs' + '_' + str(mark.year) + '0' + str (mark.month) + str(mark.day) + '.csv'
att_file = 'DeviceLogs_20180628.csv'

#Biometric Log details headers
Logs = pd.read_csv(att_file)
Logs.head()

,DeviceLogId,DownloadDate,DeviceId,UserId,LogDate,Direction,AttDirection,C1,C2,C3,C4,C5,C6,C7,WorkCode,UpdateFlag
0,1,06/02/18 05:00:00,14.0,102,06/01/18 07:57:03,NaN,NaN,in,NaN,NaN,0,0,NaN,NaN,0,0.0
1,2,06/02/18 05:00:00,14.0,73,06/01/18 07:58:30,NaN,NaN,in,NaN,NaN,0,0,NaN,NaN,0,0.0
2,3,06/02/18 05:00:00,14.0,74,06/01/18 07:58:37,NaN,NaN,in,NaN,NaN,0,0,NaN,NaN,0,0.0
3,4,06/02/18 05:00:00,14.0,500,06/01/18 08:17:01,NaN,NaN,in,NaN,NaN,0,0,NaN,NaN,0,0.0
4,5,06/02/18 05:00:00,14.0,59,06/01/18 08:59:29,NaN,NaN,in,NaN,NaN,0,0,NaN,NaN,0,0.0


In [111]:
#Good Morning
print('Good Morning, MJ. \nAttendance File sent!')
att_date = input('What date attendance you want to upload: ')
att_date

Good Morning, MJ. 
Attendance File sent!
What date attendance you want to upload: 27/06/2018


'27/06/2018'

In [112]:
#find the useful columns from the attendance sheet
csvTA = (pd.read_csv(att_file)).loc[ : ,['UserId', 'LogDate']]
csvTA.head()

,UserId,LogDate
0,102,06/01/18 07:57:03
1,73,06/01/18 07:58:30
2,74,06/01/18 07:58:37
3,500,06/01/18 08:17:01
4,59,06/01/18 08:59:29


In [113]:
#Define TA dataframe
#Split LogDate columns with naming them
TA = pd.DataFrame()
TA[['dates', 'Times']] = pd.DataFrame([x.split(' ') for x in csvTA.LogDate.tolist()])
TA.head()

,dates,Times
0,06/01/18,07:57:03
1,06/01/18,07:58:30
2,06/01/18,07:58:37
3,06/01/18,08:17:01
4,06/01/18,08:59:29


In [114]:
#change the date format
TA['date'] = pd.to_datetime(TA['dates'], format = '%m/%d/%y').dt.strftime('%d/%m/%Y')
TA = TA.drop('dates', axis =1)
TA.head()

,Times,date
0,07:57:03,01/06/2018
1,07:58:30,01/06/2018
2,07:58:37,01/06/2018
3,08:17:01,01/06/2018
4,08:59:29,01/06/2018


In [115]:
#merging the two UserId Series with obtained DataFrame
#Do focus on axis attribute
#choose the concerned dates only and remove the machine entries
sheet = pd.concat([csvTA.UserId, TA], axis=1)
sheet = sheet.loc[(sheet.date == att_date) & (sheet.UserId != 500)].sort_values(by = 'UserId')
sheet.head()

,UserId,Times,date
1564,3,22:05:28,27/06/2018
1497,3,09:43:37,27/06/2018
1496,8,09:41:49,27/06/2018
1544,8,19:12:13,27/06/2018
1505,18,10:11:32,27/06/2018


In [116]:
#change the time format
sheet['Time'] = pd.to_datetime(sheet['Times'], format = '%H:%M:%S').dt.strftime('%H:%M')
sheet = sheet.drop('Times', axis =1)
sheet.head()

,UserId,date,Time
1564,3,27/06/2018,22:05
1497,3,27/06/2018,09:43
1496,8,27/06/2018,09:41
1544,8,27/06/2018,19:12
1505,18,27/06/2018,10:11


In [117]:
#grouping and aggregating max min values to find out the In and Out time
final = pd.DataFrame(sheet.groupby(['UserId', 'date']).Time.agg([min, max])).reset_index()

#renaming columns to the required format
final.columns.values[0] = 'personal_profile_employee_code'
final.columns.values[2] = 'in_time'
final.columns.values[3] = 'out_time'

final.head()

,personal_profile_employee_code,date,in_time,out_time
0,3,27/06/2018,09:43,22:05
1,8,27/06/2018,09:41,19:12
2,18,27/06/2018,10:11,19:04
3,20,27/06/2018,08:54,23:10
4,23,27/06/2018,10:51,19:47


In [119]:
# write csv

final.to_csv('time_attendance.csv', index = False)
print('Hallelujuah!')

Hallelujuah!
